In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

dataset = pd.read_table('Data/data_banknote_authentication.txt', sep = ",", header=None)
X = dataset.iloc[:, :-1].values
t = dataset.iloc[:, -1].values

# split the training set, validation set and test set of ratio 6:2:2
X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=8775)
X_train, X_valid, t_train, t_valid = train_test_split(X_train, t_train, test_size=0.25, random_state=8775)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
X_valid = sc.transform(X_valid)


In [ ]:
def ReLU(x):
    return x * (x > 0)

def d_ReLU(x):
    return 1. * (x > 0)

def standard_normal(matrix):    # mean=0, stdev=1
    return np.random.standard_normal(size=matrix.shape)

def random_integers(matrix):
    opt = np.random.choice([0,1],size=matrix.shape)
    return np.where(opt==0, -1, opt)

def pattern(matrix):
    for m in range (matrix.shape[0]):

        if np.matrix.ndim != 1:
            for n in range (matrix.shape[1]):
                matrix[m][n] = (m+n)%2
        else:
            matrix[m] = m%2

    return np.where(matrix==0, -1, matrix)

def cross_entropy(y,t):
    eps = np.finfo(float).eps
    cross_entropy = -np.sum(t * np.log(y + eps))
    return cross_entropy


def NNClassifier(X, t, hidden_layer_sizes, initilize, epochs, learning_rate):
    layer_1_size = hidden_layer_sizes[0]
    layer_2_size = hidden_layer_sizes[1]
    
    output = np.zeros(len(X))

    w_1 = np.ones((layer_1_size, 5))
    w_2 = np.ones((layer_2_size, layer_1_size+1))
    w_3 = np.ones((1, layer_2_size+1))

    w_1 = initilize(w_1)
    w_2 = initilize(w_2)
    w_3 = initilize(w_3)

    print(w_3)

    j=0

    for i in range (len(X)):
        input = X[i]

        # forward pass
        z_1 = np.dot(w_1, np.insert(input,0,1).T)
        h_1 = ReLU(z_1)

        z_2 = np.dot(w_2, np.insert(h_1,0,1).T)
        h_2 = ReLU(z_2)

        z_3 = np.dot(w_3, np.insert(h_2,0,1).T)

        output[i] = np.rint( np.power((1 + np.exp(-z_3)), -1) )

        
        while(j<=epochs):

            dz_3 = -output[i]+ np.power((1 + np.exp(-z_3)), -1)
            gw_3 = dz_3*np.insert(h_2.T,0,1)
            gz_2 = np.multiply(d_ReLU(z_2),np.dot( np.delete(w_3,0,1).T, dz_3))

            gw_2 = gz_2.reshape(layer_2_size,1)*np.insert(h_1.T,0,1)
            gz_1 = np.multiply(d_ReLU(z_1),np.dot( np.delete(w_2,0,1).T, gz_2))

            gw_1 = gz_1.reshape(layer_1_size,1)*np.insert(input.T,0,1)

            # update w
            w_3 = w_3-learning_rate*gw_3
            w_2 = w_2-learning_rate*gw_2
            w_1 = w_1-learning_rate*gw_1

            j=j+1

    loss = cross_entropy(output,t)

    return output, loss

In [ ]:
o, l=NNClassifier(X_train, t_train,hidden_layer_sizes=(30,50), initilize = random_integers, epochs=50,learning_rate=0.005)
print(l)
